План работы над уроком:
- Ради фана - использовать даты из Назад в Будущее - 19 часов 28 минут 21 октября 2015 года - подобрать с помощью strptime соответствующие `struct_time`. Сделать папку с уроком, чтобы ссылаться на упражнения.
- Дать ссылки на документации модулей - в описании библиотек
- Модуль timeit
- Раздел о таймбарах и схожих элементах - в командной строке и jupyter progressbar, tqdm
- perf_counter()
- time_ns() - начиная с 3.7
- time.pthread_getcpuclockid(thread_id)
- в конце работы - опписать содержание блокнота для удобства перемещения по нему
- astropy и подобные - просто ссылки с краткими пояснениями

В этом блокноте я постарался обобщить сведения по работе с различными библиотеками Python, оперирующими временем, временными метками и т.п.

# Назад в будущее: практическое руководство по перемещению во времени с Python


В Python несколько библиотек для работы со временем и интервалами времени: time, datatime, calendar, timeit. Но когда какую бибиотеку использовать? Подробно объясняем на примерах.

Подготовлено при поддержке [proglib](https://proglib.io/).

Функции, связанные со временем, распределены во встроенных модулях Python 3 довольно компактно. Однако наблюдаются и некоторые пересечения. Для начала импортируем встроенные библиотеки:

In [1]:
import time
import timeit
import datetime
import calendar

Вам нужен модуль `time`, если:
- код оперирует секундами, или еще меньшими отрезками времени ( милли-, микро-, наносекундами)
- нужно обрабатывать вывод и считывание строк, содержащих элементы времени, например, строки вида `Nov 12 1955 06:38`
- нужна программная остановка выполнения кода программы на определенное время

Вам нужен модуль `datetime`, если:
- Если ваш код предполагает большой объем работы с датами, разумнее использовать встроенный модуль `datetime`.

# 1. Работа с временной шкалой: модуль time

## 1.1. Понятие epoch

Работа с модулем time в существенной мере зависит от операционной системы, так как время привязано к фиксированной начальной точке –  эпохе (epoch). Чтобы узнать эту начальную точку:

In [2]:
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)

Например, в Unix-системах мы получим **объект** `time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)`. То есть точкой отсчета в этом случае является 1 января 1970 г.

Время, прошедшее с начала эпохи отображается в **секундах** (`seconds since the epoch`):

In [3]:
В каких случаях используетсяtime.time()

1574953382.7262268

Так как понятие эпохи для разных систем может отличаться, на разных операционных системах число секунд, возвращаемое функцией `time()` может быть различным.

Время до точки `epoch` тоже существует, но значения секунд `seconds since the epoch` отрицательны:

In [4]:
time.gmtime(-10**8)

time.struct_time(tm_year=1966, tm_mon=10, tm_mday=31, tm_hour=14, tm_min=13, tm_sec=20, tm_wday=0, tm_yday=304, tm_isdst=0)

## 1.2. Секунды, struct_time и преобразование друг в друга

![title](img/Clock.png)

Как видно, из предыдущего раздела, модуль `time` оперирует двумя основными типами объектов: `struct_time` и секундами с начала эпохи. Для преобразований от одного типа к другому имеются следующие функции:

- `gmtime()`: секунды в `struct_time` UTC 
- `localtime()`: секунды в `struct_time` для местного времени 
- `calendar.timegm()` (не модуль time): struct_time UTC в секунды 
- `mktime()`: `struct_time` локального времени в секунды 

UTC – стандартное обозначение [всемирного координированного времени](https://ru.wikipedia.org/wiki/%D0%92%D1%81%D0%B5%D0%BC%D0%B8%D1%80%D0%BD%D0%BE%D0%B5_%D0%BA%D0%BE%D0%BE%D1%80%D0%B4%D0%B8%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D0%BE%D0%B5_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F) без учета [часовых поясов](https://ru.wikipedia.org/wiki/%D0%A7%D0%B0%D1%81%D0%BE%D0%B2%D0%BE%D0%B9_%D0%BF%D0%BE%D1%8F%D1%81). Начало отсчета `epoch` привязано к UTC, то есть не зависит от местного времени.

In [5]:
time.gmtime(1574869797)

time.struct_time(tm_year=2019, tm_mon=11, tm_mday=27, tm_hour=15, tm_min=49, tm_sec=57, tm_wday=2, tm_yday=331, tm_isdst=0)

In [6]:
time.localtime(1574869797)  # заметьте отличие в выводе значения tm_hour

time.struct_time(tm_year=2019, tm_mon=11, tm_mday=27, tm_hour=18, tm_min=49, tm_sec=57, tm_wday=2, tm_yday=331, tm_isdst=0)

В отсутствии аргумента функции `gmtime` и `localtime` возвращают значение для текущего времени (соответственно UTC и местное время).

Для преобразования `struct_time` в секунды, можно или передать сам объект, или кортеж целых чисел. Кортеж элементов соответствует тому же порядку:

1. Год `tm_year`
2. Месяц `tm_mon` – целое число (`1` - Январь, `12` - Декабрь)
3. День месяца `tm_day`
4. Час `tm_hour` – целое число в диапазоне от `0` до `23`
5. Минута `tm_min`
6. Секунда `tm_sec`
7. День недели `tm_wday` – целое число от `0` (Понедельник) до `6` (Воскресенье)
8. День года `tm_yday`
9. Целочисленный флаг `tm_isdst` для учета перехода на [летнее время](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D1%82%D0%BD%D0%B5%D0%B5_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F) (daylight saving time, DST): `1` - переход на летнее время учитывается, `0` - не учитывается, `-1` - неизвестно

In [7]:
time.mktime((2019, 11, 27, 15, 49, 57, 2, 331, 0))  # значения соответствуют кортежу localtime

1574858997.0

In [8]:
time.mktime(time.localtime())

1574953382.0

Для перевода не местного, а международного времени в секунды необходимо использовать модуль `calendar`:

In [9]:
calendar.timegm(time.gmtime())

1574953382

Заметим, что в отличие от представления в виде секунд, ни `struct_time`, ни соответствующий кортеж, не предполагают отображения значений времени, меньших, чем секунда.

Удобство применения `struct_time` заключается в том, что его структура соответствует [именнованному кортежу](https://docs.python.org/3/library/collections.html#collections.namedtuple). Это позволяет писать более ясный код: элементы объекта можно вызывать независимо по меткам с  говорящими названиями вместо индексов:

In [10]:
t = time.localtime()
'Итак, на дворе {}-й год.'.format(t.tm_year)

'Итак, на дворе 2019-й год.'

Кроме вышеперечисленных параметров, `struct_time` содержит дополнительные. Важно помнить, что кроме часовых поясов, в каждой отдельной стране местным законодательством регулируется переход на летнее время. Начиная с версии Python 3.6, узнать действует ли сейчас летнее время (в России в 2014 году произведен переход на [постоянное «зимнее» время](https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D1%82%D0%BD%D0%B5%D0%B5_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F#%D0%9F%D0%B5%D1%80%D0%B5%D1%85%D0%BE%D0%B4_%D0%BD%D0%B0_%D0%BF%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%BD%D0%BE%D0%B5_%C2%AB%D0%B7%D0%B8%D0%BC%D0%BD%D0%B5%D0%B5%C2%BB_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F):

In [11]:
t.tm_isdst

0

Считаем часовой пояс:

In [12]:
t.tm_zone

'MSK'

Смещение местного времени относительно UTC в секундах:

In [13]:
t.tm_gmtoff

10800

## 1.3. Строки временных меток

![](img/Newspaper.png)

Популярно также преобразование типов в строки вида `timestamp`, например, `Tue Nov 26 18:30:20 2019`. Для этого применяются функции `ctime()` и `asctime()`:
- `ctime()` – принимает время в секундах
- `asctime()` – принимает `struct_time` (по умолчанию используется местное время)

In [14]:
time.ctime(time.time())  # преобразует время в секундах в timestamp для местного времени

'Thu Nov 28 18:03:02 2019'

In [15]:
time.asctime()  # Аналогично time.asctime(time.localtime())

'Thu Nov 28 18:03:02 2019'

In [16]:
time.asctime(time.gmtime())  # Время UTC

'Thu Nov 28 15:03:02 2019'

Хотя строковый вывод функций `ctime()` и `asctime()` довольно удобен, может потребоваться альтернативный формат. Для гибкого форматирования в библиотеку `time` включена функция `strftime()`. Функция принимает строку шаблона форматирования со спецификаторами и сам объект времени.

In [17]:
time.strftime('%d.%m.%Y', time.localtime())

'28.11.2019'

Функция `strftime()` также удобна для автоматической локализации строк.

In [18]:
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.utf8')
time.strftime('Текущее время: %c', time.localtime())

'Текущее время: Чт 28 ноя 2019 18:03:02'

Список [спецификаторов шаблона](https://docs.python.org/3/library/time.html#time.strftime):
- `%a`, `%A` – аббревиатура и полное название дня недели (`Чт`, `Четверг`)
- `%b`, `%B` – то же для месяца с учетом склонения (`ноя`, `ноября`)
- `%с` – локализованная строка временной метки
- `%d` – день месяца (`28`)
- `%H`, `%I` – Час в 24- и 12-часовом представлении (`17`, `05`)
- `%j` – номер дня года (в представлении от `001` до `366`)
- `%m` – двузначное представление месяца (от `01` до `12`)
- `%M` – двузначное представление минут (от `00` до `59`)
- `%p` – местный эквивалент AM и PM
- `%S` – двузначное представление секунд
- `%W` – двузначное представление номера недели, первый день – Пн (`%U` для Вс)
- `%w` – двузначное представление номера дня недели
- `%x`, `%X` – принятый способ представления даты и времени.
- `%y`, `%Y` – двуназначное (без века) и четырехзначное представление года
- `%z`, `%Z` – обозначение часового пояса в четырехзначном формате со знаком плюс или минус и в виде названия часового пояса

Пример одновременного использования нескольких спецификаторов:

In [19]:
s = """Сегодня %A, %d %B. В России эту дату обычно записывают
следующим образом: %x или сокращенно: %d.%m.%y.
Это %j день года, %W неделя. На часах %X.
Часовой пояс: %Z."""

print(time.strftime(s, time.localtime()))

Сегодня Четверг, 28 ноября. В России эту дату обычно записывают
следующим образом: 28.11.2019 или сокращенно: 28.11.19.
Это 332 день года, 47 неделя. На часах 18:03:02.
Часовой пояс: MSK.


Что, если у нас есть строка, содержащая метку времени, а мы хотим распарсить ее в объект `struct_time`, чтобы обработать его в Python? Для этого есть функция `strptime()`. Первый аргумент – строка, второй – правило, описанное через те же спецификаторы:

In [22]:
time.strptime('29.11.19', '%d.%m.%y')

time.struct_time(tm_year=2019, tm_mon=11, tm_mday=29, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=333, tm_isdst=-1)

Функция `strptime()` в частности позволяет кратко задавать `struct_time`, не используя все девять позиций кортежа. Неизвестные элементы будут вычислены или на их место будут подставлены значения по умолчанию.

## 1.4. Приостановка выполнения кода и оценка производительности

![title](img/Clock2.jpg)

Одна из наиболее часто используемых функций модуля `time` – функция `sleep()`, выполняющая задержку исполнения программного кода на переданное число секунд (можно использовать дробные значения):

In [30]:
print(time.strftime('Текущее время: %X.'))
print('Задержка...')
time.sleep(5)
print('Прошло время.')
print(time.strftime('Текущее время: %X.'))

Текущее время: 19:32:36.
Задержка...
Прошло время.
Текущее время: 19:32:41.


Функция `sleep()` нередко используется для тестирования кода, намеренного внесения задержек на различных этапах работы кода.

Для оценки производительности однократно запускаемых команд, можно использовать функцию `perf_counter()`, которая обеспечивает лучшее разрешение по времени на коротких интервалов времени, нежеле функция `time`:

In [49]:
def longrunning_function():
    for i in range(3):
        time.sleep(1)
        
def shortrunning_function():
    n = 1
    for i in range(2, 100):
        n *= i

start = time.perf_counter()
longrunning_function()
end = time.perf_counter()
print("Выполнение longrunning_function() заняло {} c.".format(end-start))

start = time.perf_counter()
shortrunning_function()
end = time.perf_counter()
print("Выполнение shortrunning_function() заняло {} c.".format(end-start))

Выполнение longrunning_function() заняло 3.0042153210015385 c.
Выполнение shortrunning_function() заняло 0.0002610590017866343 c.


В версиии 3.7 (и выше) добавлена функция `perf_counter_ns()` – работает так же, но время выводит в наносекундах, что удобнее для совсем малых интервалов времени и быстро исполняемых команд.

Однако, если вы пытаетесь точно измерить производительность фрагмента кода, мы рекомендуем использовать модуль `timeit`.

# 2. Оценка производительности: timeit

![title](img/DocRemoteControl.jpg)

# 3. Работа с датами: datetime

![title](img/MartyTime.jpg)

Когда использовать модуль:
- Если вам важны даты, а не секунды
- Если вам важны интервалы между отметками времени разных дат
документация библиотеки

## 3.Х. Интервалы времени timedelta

# 4. Работа с календарем: calendar

# 5. Другие встроенные модули, связанные со временем
![](ClockTower.pg)

# 6. Сторонние библиотеки
![title](img/Weird.jpg)

## 7. Таймбары

![](img/Bar.jpeg)

# Резюме

Источники

- https://realpython.com/python-time-module/